In [ ]:
from importlib.resources import path
import os,io
from google.cloud import vision
from google.cloud import vision_v1
from google.cloud.vision_v1 import types
import pandas as pd
from tqdm import tqdm
import cv2

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/kaggle/input/cloudvisionapi-json/manuscripts-430205-87e76d29241b.json"

In [ ]:
client = vision.ImageAnnotatorClient()

In [ ]:
def detectText(img_path):
    with io.open(img_path, 'rb') as image_file:
        content = image_file.read()
    
    # Set the image context for handwritten Sanskrit
    image = vision_v1.types.Image(content=content)
    # image_context = vision_v1.types.ImageContext(language_hints=['sa'])  # 'sa' is the language code for Sanskrit
    
    # Perform text detection with image context
    response = client.document_text_detection(image=image)
    texts = response.text_annotations
    
    # Initialize an empty DataFrame
    df = pd.DataFrame(columns=['locale', 'description'])
    
    # Collect data into a list of dictionaries
    data = []
    for text in texts:
        data.append({
            'locale': text.locale,
            'description': text.description
        })
    
    # Create a DataFrame from the collected data
    new_df = pd.DataFrame(data)
    
    # Append the new_df to the existing df using pd.concat
    df = pd.concat([df, new_df], ignore_index=True)
    
    return texts, df

def process_folder(folder_path):
    all_texts_df = pd.DataFrame(columns=['locale', 'description', 'filename'])
    
    # Iterate over all files in the folder
    for filename in tqdm(os.listdir(folder_path), desc="Processing images"):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            # Perform OCR on the image
            texts, df = detectText(image_path)
            df['filename'] = filename  # Add filename to the DataFrame
            
            # Append to the overall DataFrame
            all_texts_df = pd.concat([all_texts_df, df], ignore_index=True)
            
            # Display filename
            print(f"Filename: {filename}")
            
            # Display the image with bounding boxes
            visualize_ocr(image_path, texts)
            
            # Print detected text
            print("Detected Text:")
            for text in texts:
                print(text.description)
    
    return all_texts_df

def visualize_ocr(image_path, texts):
    image = cv2.imread(image_path)
    for text in texts:
        vertices = [(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices]
        pts = np.array(vertices, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=2)
    
    # Convert image from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display the image
    plt.figure(figsize=(12, 12))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Folder containing the images
FOLDER_PATH = "/kaggle/input/sans-samplev2"

# Process the folder and get all texts
all_texts_df = process_folder(FOLDER_PATH)

# Print the DataFrame with all detected texts
print(all_texts_df)